In [1]:
using QuantEcon
using BasisMatrices
using Optim
using Interpolations
using Plots

In [2]:
struct NonrenewableResource
    a1::Float64 # inverse demand function parameter
    a2::Float64 # inverse demand function parameter
    b1::Float64 # cost function parameter
    b2::Float64 # cost function parameter
    delta::Float64 # Discount factor
    s_vec::Vector{Float64} # ore grid
end

In [3]:
n = 101
smin = 0
smax = 10
fspace = Basis(ChebParams(n, smin, smax))
snodes = nodes(fspace)

([0.0, 0.0333333, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8  …  9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9, 9.96667, 10.0], Array{Float64,1}[[0.0, 0.0333333, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8  …  9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9, 9.96667, 10.0]])

In [4]:
NR = NonrenewableResource(10,0.8,12,1.0,0.9,snodes[1])

NonrenewableResource(10.0, 0.8, 12.0, 1.0, 0.9, [0.0, 0.0333333, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8  …  9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9, 9.96667, 10.0])

In [8]:
# 補間の準備,3次のスプラインで補間する
fspace = Basis(SplineParams(n,smin,smax,3))
s_vec,_ = nodes(fspace)
Φ = BasisMatrix(fspace, Expanded(), s_vec, 0)
coeffs_V = Φ.vals[1] \ s_vec;

In [ ]:
function update_Bellman(NR::NonrenewableResource,V::Vector)
    
    a1,a2,b1,b2,delta = NR.a1,NR.a2,NR.b1,NR.b2,NR.delta
    V_new = similar(V)
    x_opt = similar(V)
    
    # 補間の準備.３次のスプラインで補間する
    fspace = Basis(SplineParams(n,smin,smax,3))
    s_vec,_ = nodes(fspace)
    Φ = BasisMatrix(fspace, Expanded(), s_vec, 0)
    coeffs_V = Φ.vals[1] \ s_vec
    
    for (s_idx,s) in enumerate(NR.s_vec)
        objective(x) = -((a1-a2*x)*x-(b1*x-0.5*b2*x*(2s-x))+delta*(funeval(coeffs_V,fspace,s-x)))
        opt = optimize(objective, 1e-10, s)
        V_new[s_idx] = - opt.minimum
        x_opt1[s_idx] = opt.minimizer
    end
    
    for (s_idx,s)
    
    return V_new,x_opt
    
end
#objective(x) = -((a1-a2*x)*x-(b1*x-0.5*b2*x*(2s-x))+delta*V_func.(s-x))

In [ ]:
V2 = Vector{Float64}(length(NR.s_vec))
for i in 1:length(NR.s_vec)
    V2[i] = V[i]/NR.s_vec[i]
end
V2
plot(NR.s_vec,V2,label="Available Stock",ylabel="Value",xlim=(0,10),ylim=(-1,6))